# Horovod with Petastorm

[Petastorm](https://github.com/uber/petastorm) enables single machine or distributed training and evaluation of deep learning models from datasets in Apache Parquet format. It supports ML frameworks such as TensorFlow, Pytorch, and PySpark and can be used from pure Python code.

**Required Libraries**: 
* `petastorm==0.8.2` via PyPI

Run the following cell to set up our environment.

In [0]:
%run "./Includes/Classroom-Setup"

## Load data

In [0]:
from sklearn.datasets.california_housing import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
np.random.seed(0)

cal_housing = fetch_california_housing()

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                        cal_housing.target,
                                                        test_size=0.2,
                                                        random_state=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /root/scikit_learn_data

## Spark DataFrame

Let's concatenate our features and label, then create a Spark DataFrame from our Pandas DataFrame.

In [0]:
data = pd.concat([pd.DataFrame(X_train, columns=cal_housing.feature_names), pd.DataFrame(y_train, columns=["label"])], axis=1)
trainDF = spark.createDataFrame(data)
display(trainDF)

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,label
-0.3623260524334147,1.8589029704653197,-0.1687733398561354,-0.16841537196793502,-0.48611380327883963,0.0019013873805031264,0.9722904618294769,-1.4225094202651083,2.556
-0.14102329270609887,1.0643482290725061,-0.206655229462896,-0.28098122921782215,-0.40424307590599845,0.00945496150012646,1.0845962598673802,-1.382659193976686,1.146
-0.6614495628340722,-1.0014940985488086,1.1065836102381363,1.1181306865387701,-1.2505372251622158,0.020553720106354596,1.0611992186094819,-0.8297373042249129,1.375
-1.2486473058563599,0.4287044359582555,-0.6256134552400134,0.09844715719762903,1.8801281976383866,-0.04435374816746968,-0.7544111830032698,0.6098521204441189,1.188
1.8289201934284547,0.7465263325153809,0.17362344642713573,-0.08335226715805534,-0.6009108014429322,-0.06091511960570208,-0.8152434902738014,0.5899270072999078,4.227
-1.4284491897150797,-0.2863948312952766,-0.7472267955849092,-0.1547632542196419,-0.5617552361776603,-0.08622743569328611,2.446304061076944,-2.2494016157497407,0.75
0.9369950000793041,-0.20693935715599523,0.8374018880054728,-0.12086910458833956,-0.6872310248686453,-0.027269637898943263,1.374719571465293,-0.8895126436575392,1.295
-0.8653589618470336,-1.0014940985488086,-0.012187847650116622,0.16510770950364012,0.3672995396619726,0.026621673885657098,-1.0866491688653979,1.3072310804913962,1.17
-0.6825965059518282,-0.7631276761309647,-0.6575323153922634,-0.08222856221328118,5.234158322293153,-0.030005757851739524,-0.6561436097201082,0.555058059297547,1.454
0.09851870546028144,0.1903380135404115,0.02467968066378206,-0.06345533690656746,-0.06163187983486943,0.06156892513144482,-0.8479993480348564,0.779215582169882,2.259


## Create Dense Vectors for Features

In [0]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=cal_housing.feature_names, outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF).select("features", "label")
display(vecTrainDF)

features,label
"List(1, 8, List(), List(-0.3623260524334147, 1.8589029704653197, -0.1687733398561354, -0.16841537196793502, -0.48611380327883963, 0.0019013873805031264, 0.9722904618294769, -1.4225094202651083))",2.556
"List(1, 8, List(), List(-0.14102329270609887, 1.0643482290725061, -0.206655229462896, -0.28098122921782215, -0.40424307590599845, 0.00945496150012646, 1.0845962598673802, -1.382659193976686))",1.146
"List(1, 8, List(), List(-0.6614495628340722, -1.0014940985488086, 1.1065836102381363, 1.1181306865387701, -1.2505372251622158, 0.020553720106354596, 1.0611992186094819, -0.8297373042249129))",1.375
"List(1, 8, List(), List(-1.2486473058563599, 0.4287044359582555, -0.6256134552400134, 0.09844715719762903, 1.8801281976383866, -0.04435374816746968, -0.7544111830032698, 0.6098521204441189))",1.188
"List(1, 8, List(), List(1.8289201934284547, 0.7465263325153809, 0.17362344642713573, -0.08335226715805534, -0.6009108014429322, -0.06091511960570208, -0.8152434902738014, 0.5899270072999078))",4.227
"List(1, 8, List(), List(-1.4284491897150797, -0.2863948312952766, -0.7472267955849092, -0.1547632542196419, -0.5617552361776603, -0.08622743569328611, 2.446304061076944, -2.2494016157497407))",0.75
"List(1, 8, List(), List(0.9369950000793041, -0.20693935715599523, 0.8374018880054728, -0.12086910458833956, -0.6872310248686453, -0.027269637898943263, 1.374719571465293, -0.8895126436575392))",1.295
"List(1, 8, List(), List(-0.8653589618470336, -1.0014940985488086, -0.012187847650116622, 0.16510770950364012, 0.3672995396619726, 0.026621673885657098, -1.0866491688653979, 1.3072310804913962))",1.17
"List(1, 8, List(), List(-0.6825965059518282, -0.7631276761309647, -0.6575323153922634, -0.08222856221328118, 5.234158322293153, -0.030005757851739524, -0.6561436097201082, 0.555058059297547))",1.454
"List(1, 8, List(), List(0.09851870546028144, 0.1903380135404115, 0.02467968066378206, -0.06345533690656746, -0.06163187983486943, 0.06156892513144482, -0.8479993480348564, 0.779215582169882))",2.259


## Array

Petastorm requires an Array as input, not a Vector. Let's register a UDF in Scala and invoke it from Python for optimal performance.

In [0]:
%scala
import org.apache.spark.ml.linalg.Vector
val toArray = udf { v: Vector => v.toArray }
spark.udf.register("toArray", toArray)

import org.apache.spark.ml.linalg.Vector
toArray: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5166/1695806283@7ca4dd8c,ArrayType(DoubleType,false),List(Some(class[value[0]: vector])),None,true,true)
res0: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5166/1695806283@7ca4dd8c,ArrayType(DoubleType,false),List(Some(class[value[0]: vector])),None,true,true)

## Save Data 

Let's write our DataFrame out as a parquet files to DBFS.

In [0]:
workingDir = "dbfs:/pinky.gtm@mail.kmutt.ac.th"
file_path = f"{workingDir}/deep-learning/petastorm.parquet"
vecTrainDF.selectExpr("toArray(features) AS features", "label").repartition(8).write.mode("overwrite").parquet(file_path)

## Remove committed/started metadata

Petastorm + Horovod do not work if you leave the committed/started metadata files in our Parquet folder. We will need to remove them.

In [0]:
[dbutils.fs.rm(i.path) for i in dbutils.fs.ls(file_path) if ("_committed_" in i.name) | ("_started_" in i.name)]

display(dbutils.fs.ls(file_path))

path,name,size
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/_SUCCESS,_SUCCESS,0
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/part-00000-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-22-1-c000.snappy.parquet,part-00000-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-22-1-c000.snappy.parquet,122253
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/part-00001-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-23-1-c000.snappy.parquet,part-00001-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-23-1-c000.snappy.parquet,121772
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/part-00002-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-24-1-c000.snappy.parquet,part-00002-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-24-1-c000.snappy.parquet,122049
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/part-00003-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-25-1-c000.snappy.parquet,part-00003-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-25-1-c000.snappy.parquet,122169
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/part-00004-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-26-1-c000.snappy.parquet,part-00004-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-26-1-c000.snappy.parquet,121958
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/part-00005-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-27-1-c000.snappy.parquet,part-00005-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-27-1-c000.snappy.parquet,122356
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/part-00006-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-28-1-c000.snappy.parquet,part-00006-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-28-1-c000.snappy.parquet,121418
dbfs:/pinky.gtm@mail.kmutt.ac.th/deep-learning/petastorm.parquet/part-00007-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-29-1-c000.snappy.parquet,part-00007-tid-8151245934285474382-83834c3d-e2f7-4921-9223-cbe4716500b1-29-1-c000.snappy.parquet,122205


## Define Model

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
# tf.set_random_seed(42)

def build_model():
  from tensorflow.keras import models, layers
  model = models.Sequential()
  model.add(layers.Dense(20, input_dim=8, activation='relu'))
  model.add(layers.Dense(20, activation='relu'))
  model.add(layers.Dense(1, activation='linear'))
  return model

## Single Node

Define shape of the input tensor and output tensor and fit the model (on the driver). We need to use Petastorm's [make_batch_reader](https://petastorm.readthedocs.io/en/latest/api.html#petastorm.reader.make_batch_reader) to create an instance of Reader for reading batches out of a non-Petastorm Parquet store.

In [0]:
from petastorm import make_batch_reader
from petastorm.tf_utils import make_petastorm_dataset

abs_file_path = file_path.replace("dbfs:/", "/dbfs/")

with make_batch_reader("file://" + abs_file_path, num_epochs=None) as reader: 
  dataset = make_petastorm_dataset(reader).map(lambda x: (tf.reshape(x.features, [-1,8]), tf.reshape(x.label, [-1,1])))
  model = build_model()
  optimizer = keras.optimizers.Adam(lr=0.001)
  model.compile(optimizer=optimizer,
                loss='mse',
                metrics=['mse'])
  model.fit(dataset, steps_per_epoch=10, epochs=10)

Epoch 1/10
 1/10 [==>...........................] - ETA: 0s - loss: 3.3908 - mse: 3.3908 6/10 [=================>............] - ETA: 0s - loss: 3.2557 - mse: 3.255710/10 [==============================] - 0s 10ms/step - loss: 3.1112 - mse: 3.1112
Epoch 2/10
 1/10 [==>...........................] - ETA: 0s - loss: 2.7343 - mse: 2.7343 3/10 [========>.....................] - ETA: 0s - loss: 2.7244 - mse: 2.7244 8/10 [=======================>......] - ETA: 0s - loss: 2.5245 - mse: 2.524510/10 [==============================] - 0s 37ms/step - loss: 2.4796 - mse: 2.4796
Epoch 3/10
 1/10 [==>...........................] - ETA: 0s - loss: 2.1512 - mse: 2.1512 6/10 [=================>............] - ETA: 0s - loss: 1.9656 - mse: 1.9656 9/10 [==========================>...] - ETA: 0s - loss: 1.8851 - mse: 1.885110/10 [==============================] - 0s 13ms/step - loss: 1.8696 - mse: 1.8696
Epoch 4/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.9793 - mse: 1.9793 7/10 [====================>.........] - ETA: 0s - loss: 1.6753 - mse: 1.675310/10 [==============================] - 0s 8ms/step - loss: 1.6451 - mse: 1.6451
Epoch 5/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.2628 - mse: 1.2628 7/10 [====================>.........] - ETA: 0s - loss: 1.3624 - mse: 1.362410/10 [==============================] - 0s 9ms/step - loss: 1.3237 - mse: 1.3237
Epoch 6/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.0732 - mse: 1.0732 7/10 [====================>.........] - ETA: 0s - loss: 1.1261 - mse: 1.126110/10 [==============================] - 0s 9ms/step - loss: 1.1297 - mse: 1.1297
Epoch 7/10
 1/10 [==>...........................] - ETA: 0s - loss: 0.9828 - mse: 0.9828 6/10 [=================>............] - ETA: 0s - loss: 0.9942 - mse: 0.994210/10 [==============================] - ETA: 0s - loss: 0.9945 - mse: 0.994510/10 [==============================] - 0s 13ms/step - loss: 0.9945 - mse: 0.9945
Epoch 8/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.1862 - mse: 1.1862 7/10 [====================>.........] - ETA: 0s - loss: 0.9988 - mse: 0.998810/10 [==============================] - 0s 8ms/step - loss: 0.9768 - mse: 0.9768
Epoch 9/10
 1/10 [==>...........................] - ETA: 0s - loss: 0.8133 - mse: 0.8133 7/10 [====================>.........] - ETA: 0s - loss: 0.8742 - mse: 0.8742

## Horovod

Let's do the same thing, but let's add in Horovod for distributed model training.

In [0]:
import horovod.tensorflow.keras as hvd

def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  with make_batch_reader("file://" + abs_file_path, num_epochs=None, cur_shard=hvd.rank(), shard_count= hvd.size()) as reader:
    dataset = make_petastorm_dataset(reader).map(lambda x: (tf.reshape(x.features, [-1,8]), tf.reshape(x.label, [-1,1])))
    model = build_model()
    from tensorflow.keras import optimizers
    optimizer = optimizers.Adam(lr=0.001*hvd.size())
    optimizer = hvd.DistributedOptimizer(optimizer)
    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mse'])
    history = model.fit(dataset, steps_per_epoch=10, epochs=10)

## Train on driver

In [0]:
from sparkdl import HorovodRunner
hr = HorovodRunner(np=-1)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
HorovodRunner will launch Horovod jobs on the driver node. There would be resource contention if you
share the cluster with others.
The global names read or written to by the pickled function are {'make_petastorm_dataset', 'hvd', 'make_batch_reader', 'build_model', 'abs_file_path', 'tf'}.
The pickled object size is 2518 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Executing command: ['mpirun', '--allow-run-as-root', '-np', '1', '-H', 'localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_462150b498d0/id_rsa', 'bash', '/tmp/HorovodRunner_462150b498d0/launch.sh'].

[1,0]<stderr>:Failed loading Unischema from metadata in %s. Assuming the dataset was not created with Petastorm. Will try to construct from native Parquet schema.
[1,0]<stderr>:Recovering rowgroup information for the entire dataset. This can take a long time for datasets with large number of files. If this dataset was generated by Petastorm (i.e. by using "with materialize_dataset(...)") and you still see this message, this indicates that the materialization did not finish successfully.
[1,0]<stderr>:2022-03-09 10:24:59.279643: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations: AVX2 FMA
[1,0]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,0]<stderr>:2022-03-09 10:24:59.286871: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2397220000 Hz
[1,0]<stderr>:2022-03-09 10:24:59.287306: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55771638ee20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2022-03-09 10:24:59.287351: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,0]<stderr>:WARNING:tensorflow:AutoGraph could not transform <function run_training_horovod.<locals>.<lambda> at 0x7f17640b6dd0> and will run it as-is.
[1,0]<stderr>:Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
[1,0]<stderr>:Cause: Unable to locate the source code of <function run_training_horovod.<locals>.<lambda> at 0x7f17640b6dd0>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
[1,0]<stderr>:To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[1,0]<stderr>:AutoGraph could not transform <function run_training_horovod.<locals>.<lambda> at 0x7f17640b6dd0> and will run it as-is.
[1,0]<stderr>:Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
[1,0]<stderr>:Cause: Unable to locate the source code of <function run_training_ho

## Better Horovod

In [0]:
import horovod.tensorflow.keras as hvd

ml_working_path = workingDir
dbutils.fs.rm(f"{ml_working_path}/petastorm_checkpoint_weights.ckpt", True)
def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  with make_batch_reader("file://" + abs_file_path, num_epochs=None, cur_shard=hvd.rank(), shard_count=hvd.size()) as reader:
    dataset = make_petastorm_dataset(reader).map(lambda x: (tf.reshape(x.features, [-1,8]), tf.reshape(x.label, [-1,1])))
    model = build_model()
    from tensorflow.keras import optimizers
    optimizer = optimizers.Adam(lr=0.001*hvd.size())
    optimizer = hvd.DistributedOptimizer(optimizer)
    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mse'])
    checkpoint_dir = f"{ml_working_path}/petastorm_checkpoint_weights.ckpt"
    callbacks = [
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
    hvd.callbacks.MetricAverageCallback(),
    hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=10, verbose=1)
    ]

    if hvd.rank() == 0:
      callbacks.append(tf.keras.callbacks.ModelCheckpoint(checkpoint_dir, save_weights_only=True))
  
    history = model.fit(dataset, steps_per_epoch=10, epochs=10, callbacks=callbacks)

In [0]:
import horovod.tensorflow.keras as hvd
from sparkdl import HorovodRunner
hr = HorovodRunner(np=-1)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
HorovodRunner will launch Horovod jobs on the driver node. There would be resource contention if you
share the cluster with others.
The global names read or written to by the pickled function are {'make_petastorm_dataset', 'hvd', 'make_batch_reader', 'build_model', 'abs_file_path', 'tf', 'ml_working_path'}.
The pickled object size is 3129 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Executing command: ['mpirun', '--allow-run-as-root', '-np', '1', '-H', 'localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_e75cd771ba8f/id_rsa', 'bash', '/tmp/HorovodRunner_e75cd771ba8f/launch.sh'].

[1,0]<stderr>:Failed loading Unischema from metadata in %s. Assuming the dataset was not created with Petastorm. Will try to construct from native Parquet schema.
[1,0]<stderr>:Recovering rowgroup information for the entire dataset. This can take a long time for datasets with large number of files. If this dataset was generated by Petastorm (i.e. by using "with materialize_dataset(...)") and you still see this message, this indicates that the materialization did not finish successfully.
[1,0]<stderr>:2022-03-09 10:26:07.100382: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations: AVX2 FMA
[1,0]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,0]<stderr>:2022-03-09 10:26:07.107805: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2397220000 Hz
[1,0]<stderr>:2022-03-09 10:26:07.108202: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ab3c513c20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2022-03-09 10:26:07.108237: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,0]<stderr>:WARNING:tensorflow:AutoGraph could not transform <function run_training_horovod.<locals>.<lambda> at 0x7efd940cfd40> and will run it as-is.
[1,0]<stderr>:Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
[1,0]<stderr>:Cause: Unable to locate the source code of <function run_training_horovod.<locals>.<lambda> at 0x7efd940cfd40>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
[1,0]<stderr>:To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[1,0]<stderr>:AutoGraph could not transform <function run_training_horovod.<locals>.<lambda> at 0x7efd940cfd40> and will run it as-is.
[1,0]<stderr>:Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
[1,0]<stderr>:Cause: Unable to locate the source code of <funct

## Run on all workers

In [0]:
from sparkdl import HorovodRunner
hr = HorovodRunner(np=0)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
Setting np=0 is deprecated and it will be removed in the next major Databricks Runtime release.
Choosing np based on the total task slots at runtime is unreliable due to dynamic executor
registration. Please set the number of parallel processes you need explicitly.
The global names read or written to by the pickled function are {'make_petastorm_dataset', 'hvd', 'make_batch_reader', 'build_model', 'abs_file_path', 'tf', 'ml_working_path'}.
The pickled object size is 3131 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.
[1,0]<stderr>:Failed loading Unischema from metadata in %s. Assuming the dataset was not created with Petastorm. Will try to construct from native Parquet schema.
[1,1]<stderr>:Failed loading Unischema from metadata in %s. Assuming the dataset was not created with Petastorm. Will try to construct from native Parquet schema.
[1,1]<stderr>:Recovering rowgroup information for the entire dataset. This can take a long time for datasets with large number of files. If this dataset was generated by Petastorm (i.e. by using "with materialize_dataset(...)") and you still see this message, this indicates that the materialization did not finish successfully.
[1,0]<stderr>:Recovering rowgroup information for the entire dataset. This can take a long time for datasets with large number of files. If this dataset was generated by Petastorm (i.e. by using "with materialize_dataset(...)") and you still see this message, this indicates that the materialization did not finish successfully.
[1,3]<stderr>:Failed loading Unischema from metadata in %s. Assuming the dataset was not created with Petastorm. Will try to construct from native Parquet schema.
[1,2]<stderr>:Failed loading Unischema from metadata in %s. Assuming the dataset was not created with Petastorm. Will try to construct from native Parquet schema.
[1,3]<stderr>:Recovering rowgroup information for the entire dataset. This can take a long time for datasets with large number of files. If this dataset was generated by Petastorm (i.e. by using "with materialize_dataset(...)") and you still see this message, this indicates that the materialization did not finish successfully.
[1,2]<stderr>:Recovering rowgroup information for the entire dataset. This can take a long time for datasets with large number of files. If this dataset was generated by Petastorm (i.e. by using "with materialize_dataset(...)") and you still see this message, this indicates that the materialization did not finish successfully.
[1,0]<stderr>:2022-03-09 10:28:18.481984: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations: AVX2 FMA
[1,0]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,1]<stderr>:2022-03-09 10:28:18.488950: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations: AVX2 FMA
[1,1]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,0]<stderr>:2022-03-09 10:28:18.490354: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2397220000 Hz
[1,0]<stderr